In [1]:
# Data Manipulation and Preprocessing
import pandas as pd
import numpy as np

# Machine Learning and Deep Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# TensorFlow/Keras for LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Visualization
import matplotlib.pyplot as plt
import altair as alt

# Warnings
import warnings
warnings.filterwarnings('ignore')


2024-11-18 09:13:53.749220: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 09:13:53.932560: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-18 09:13:54.124233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731942834.304120    5693 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731942834.333929    5693 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 09:13:54.668309: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
#pip install tensorflow

  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 994.0 kB/s eta 0:00:0031m? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 2.1 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 12.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.2 MB/s eta 0:00:0031m14.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:0

In [1]:
#pip install altair

  Using cached altair-5.4.1-py3-none-any.whl.metadata (9.4 kB)
Using cached altair-5.4.1-py3-none-any.whl (658 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
df = pd.read_csv('/home/christopher/research_proj/cleaned_energy_weather_climate_combined.csv')

## Exploratory Data Analysis

In [4]:
# Display basic information
print("Basic Info:")
print(df.info())

# Preview data
print("\nSample Rows:")
print(df.head())

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Descriptive statistics
print("\nDescriptive Statistics:")
print(df.describe(include='all'))

# Check unique counts for categorical columns
print("\nUnique Values:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].nunique()} unique values")

Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823057 entries, 0 to 823056
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   type                823057 non-null  object 
 1   value               823057 non-null  int64  
 2   year                823057 non-null  int64  
 3   date                823057 non-null  int64  
 4   hour                823057 non-null  int64  
 5   state               823057 non-null  object 
 6   temperature         823057 non-null  float64
 7   wind_speed          823057 non-null  float64
 8   sea_level_pressure  823053 non-null  float64
 9   precip_depth_mm     823057 non-null  float64
 10  relative_humidity   823053 non-null  float64
 11  HLY-CLDH-NORMAL     822289 non-null  float64
 12  HLY-HTDH-NORMAL     822289 non-null  float64
 13  HLY-HIDX-NORMAL     822289 non-null  float64
 14  HLY-WCHL-NORMAL     822289 non-null  float64
 15  HLY-TEMP-NORMAL     82

### Let's pivot column 'type' into four categories of energy measurement and drop the 'value', and 'type' column.

In [6]:
# Pivot the 'type' and 'value' columns while keeping all other columns intact
df_pivoted = df.pivot_table(
    index=[col for col in df.columns if col not in ['type', 'value']],
    columns='type',
    values='value'
).reset_index()

# Rename the new columns for clarity (optional)
df_pivoted.columns.name = None  # Remove the column index name
df_pivoted.rename(columns={'D': 'D_energy', 'DF': 'DF_energy', 'NG': 'NG_energy', 'TI': 'TI_energy'}, inplace=True)

# Preview the resulting dataframe
print(df_pivoted.head())



   year      date  hour state  temperature  wind_speed  sea_level_pressure  \
0  2019  20190101     0    CA    49.569936    3.439894         1014.288003   
1  2019  20190101     0    FL    70.033045    3.152386         1017.141438   
2  2019  20190101     0    NY    39.318214    3.726190         1015.919618   
3  2019  20190101     0    TX    46.273627    3.590934         1014.320247   
4  2019  20190101     1    CA    48.776040    3.454301         1014.440373   

   precip_depth_mm  relative_humidity  HLY-CLDH-NORMAL  HLY-HTDH-NORMAL  \
0         0.034066          98.620430         0.009524        21.157143   
1         0.078030          92.126579         2.124074         8.181481   
2         0.000000          91.525145         0.000000        36.683333   
3         0.046001         102.676080         0.310526        20.080263   
4         0.042498         112.236430         0.007143        21.794048   

   HLY-HIDX-NORMAL  HLY-WCHL-NORMAL  HLY-TEMP-NORMAL  HLY-TEMP-10PCTL  \
0      

Checking NA values for normals

In [7]:
columns_to_check = [
    'HLY-CLDH-NORMAL', 'HLY-HTDH-NORMAL', 'HLY-HIDX-NORMAL',
    'HLY-WCHL-NORMAL', 'HLY-TEMP-NORMAL', 'HLY-TEMP-10PCTL', 'HLY-TEMP-90PCTL'
]

# Check for rows where all the specified columns are simultaneously missing
simultaneously_missing = df[columns_to_check].isnull().all(axis=1)

# Count the number of rows with simultaneous missing values
num_simultaneous_missing = simultaneously_missing.sum()

# Print results
print(f"Number of rows where all specified columns are simultaneously missing: {num_simultaneous_missing}")

# Optional: Get the indices of rows with simultaneous missing values
missing_indices = df[simultaneously_missing].index.tolist()
print(f"Indices of rows with simultaneous missing values: {missing_indices}")


Number of rows where all specified columns are simultaneously missing: 768
Indices of rows with simultaneous missing values: [24708, 24709, 24710, 24711, 24712, 24713, 24714, 24715, 24716, 24717, 24718, 24719, 24720, 24721, 24722, 24723, 24724, 24725, 24726, 24727, 24728, 24729, 24730, 24731, 24732, 24733, 24734, 24735, 24736, 24737, 24738, 24739, 24740, 24741, 24742, 24743, 24744, 24745, 24746, 24747, 24748, 24749, 24750, 24751, 24752, 24753, 24754, 24755, 24756, 24757, 24758, 24759, 24760, 24761, 24762, 24763, 24764, 24765, 24766, 24767, 24768, 24769, 24770, 24771, 24772, 24773, 24774, 24775, 24776, 24777, 24778, 24779, 24780, 24781, 24782, 24783, 24784, 24785, 24786, 24787, 24788, 24789, 24790, 24791, 24792, 24793, 24794, 24795, 24796, 24797, 24798, 24799, 24800, 24801, 24802, 24803, 164964, 164965, 164966, 164967, 164968, 164969, 164970, 164971, 164972, 164973, 164974, 164975, 164976, 164977, 164978, 164979, 164980, 164981, 164982, 164983, 164984, 164985, 164986, 164987, 164988, 16